In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#0. 패키지 설치

In [2]:
!pip install python-mecab-ko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.6/575.6 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 19.6 MB/s eta 0:00:00


#1. 유저 사전 불러오기

In [3]:
!python3 -m mecab dict-index --userdic nouns.dic '/content/drive/MyDrive/menu_labelling/dictionaries/dict3_v1_1.csv'

reading /content/drive/MyDrive/menu_labelling/박민영/dictionaries/dict3_v1_1.csv ... 398
emitting double-array: 100% |###########################################| 

done!


#2. 패키지 및 데이터 불러오기

In [4]:
import os
import pandas as pd
from mecab import MeCab
from tqdm import tqdm

In [7]:
data = pd.read_csv('/content/drive/MyDrive/menu_labelling/\'ㄷ\'으로 시작하는 데이터/d_filtered_preprocess.csv')

In [8]:
data.head()

,store,MENU_NM,표준화상품명,MENU_NM2,MENU_NM3
0,유이요거트_부천점,(다크)HOT아메리카노,아메리카노,(다크)HOT아메리카노,다크HOT아메리카노
1,유이요거트_부천점,(다크)HOT아메리카노(오픈이벤트10월20일까지),아메리카노,(다크)HOT아메리카노(오픈이벤트10월20일까지),다크HOT아메리카노오픈이벤트10월20일까지
2,유이요거트_부천점,(다크)HOT아메리카노금액 할인,아메리카노,(다크)HOT아메리카노금액 할인,다크HOT아메리카노금액 할인
3,유이요거트_부천점,(다크)ICE 아메리카노,아메리카노,(다크)ICE 아메리카노,다크ICE 아메리카노
4,유이요거트_부천점,(다크)ICE아메리카노,아메리카노,(다크)ICE아메리카노,다크ICE아메리카노


#3. mecab로 pos 태깅 진행

In [9]:
# 유저 사전 설정
mecab = MeCab(user_dictionary_path='nouns.dic')

In [10]:
# pos에서 단어만 불러오는 함수
def str_to_pos(s):
  if s!="missing":
    poslist = mecab.pos(s)
    poses = [y[0] for y in poslist]
    return poses
  else:
    return []

In [11]:
# 메뉴명에 함수 적용
data['pos'] = data['MENU_NM3'].apply(str_to_pos)

In [12]:
data.head()

,store,MENU_NM,표준화상품명,MENU_NM2,MENU_NM3,pos
0,유이요거트_부천점,(다크)HOT아메리카노,아메리카노,(다크)HOT아메리카노,다크HOT아메리카노,"[다크, HOT, 아메리카노]"
1,유이요거트_부천점,(다크)HOT아메리카노(오픈이벤트10월20일까지),아메리카노,(다크)HOT아메리카노(오픈이벤트10월20일까지),다크HOT아메리카노오픈이벤트10월20일까지,"[다크, HOT, 아메리카노, 오픈, 이, 벤트, 10, 월, 20, 일, 까지]"
2,유이요거트_부천점,(다크)HOT아메리카노금액 할인,아메리카노,(다크)HOT아메리카노금액 할인,다크HOT아메리카노금액 할인,"[다크, HOT, 아메리카노, 금액, 할인]"
3,유이요거트_부천점,(다크)ICE 아메리카노,아메리카노,(다크)ICE 아메리카노,다크ICE 아메리카노,"[다크, ICE, 아메리카노]"
4,유이요거트_부천점,(다크)ICE아메리카노,아메리카노,(다크)ICE아메리카노,다크ICE아메리카노,"[다크, ICE, 아메리카노]"


#4. 각 pos를 데이터프레임에 추가

In [13]:
maxlen = data['pos'].apply(len).max()

In [14]:
tempdf = pd.DataFrame(data['pos'].tolist(),columns=[f'Pos{i+1}' for i in range(maxlen)])

In [15]:
resdf= pd.concat([data,tempdf],axis=1)

In [16]:
resdf.head()

,store,MENU_NM,표준화상품명,MENU_NM2,MENU_NM3,pos,Pos1,Pos2,Pos3,Pos4,...,Pos9,Pos10,Pos11,Pos12,Pos13,Pos14,Pos15,Pos16,Pos17,Pos18
0,유이요거트_부천점,(다크)HOT아메리카노,아메리카노,(다크)HOT아메리카노,다크HOT아메리카노,"[다크, HOT, 아메리카노]",다크,HOT,아메리카노,None,...,None,None,None,None,None,None,None,None,None,None
1,유이요거트_부천점,(다크)HOT아메리카노(오픈이벤트10월20일까지),아메리카노,(다크)HOT아메리카노(오픈이벤트10월20일까지),다크HOT아메리카노오픈이벤트10월20일까지,"[다크, HOT, 아메리카노, 오픈, 이, 벤트, 10, 월, 20, 일, 까지]",다크,HOT,아메리카노,오픈,...,20,일,까지,None,None,None,None,None,None,None
2,유이요거트_부천점,(다크)HOT아메리카노금액 할인,아메리카노,(다크)HOT아메리카노금액 할인,다크HOT아메리카노금액 할인,"[다크, HOT, 아메리카노, 금액, 할인]",다크,HOT,아메리카노,금액,...,None,None,None,None,None,None,None,None,None,None
3,유이요거트_부천점,(다크)ICE 아메리카노,아메리카노,(다크)ICE 아메리카노,다크ICE 아메리카노,"[다크, ICE, 아메리카노]",다크,ICE,아메리카노,None,...,None,None,None,None,None,None,None,None,None,None
4,유이요거트_부천점,(다크)ICE아메리카노,아메리카노,(다크)ICE아메리카노,다크ICE아메리카노,"[다크, ICE, 아메리카노]",다크,ICE,아메리카노,None,...,None,None,None,None,None,None,None,None,None,None


In [17]:
resdf = resdf.drop('pos',axis=1)

#5. 데이터 저장

In [18]:
resdf.to_csv('/content/drive/MyDrive/menu_labelling/\'ㄷ\'으로 시작하는 데이터/d_filtered_preprocess_dict1.csv',index=False)